In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from model.model.personalized_bert import EventBertModel
from model.model.PairwiseLecardPLM1 import PairwisePLM1
from formatter.PairwiseFormatter1 import PairwiseFormatter
from config_parser import create_config
from tools.init_tool1 import init_formatter
from torch.autograd import Variable
from lime.lime_text import LimeTextExplainer
import numpy as np
# plm_path = "/home/pub/fan/SCR/Pretrain_model/bert_base_chinese"
# tokenizer = AutoTokenizer.from_pretrained(plm_path)
device = torch.device("cuda")
plt.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False  # 正确显示负号
def correct_state_dict(state_dict):
    """调整状态字典中的键名，以适应当前模型结构"""
    new_state_dict = {}
    for k, v in state_dict.items():
        # 移除所有不必要的'module.'前缀
        new_key = k.replace("module.", "")
        new_state_dict[new_key] = v
    return new_state_dict

def load_model(model_path):
    checkpoint = torch.load(model_path, map_location='cpu')
    model = PairwisePLM1()
    # 使用修正函数处理状态字典
    fixed_state_dict = correct_state_dict(checkpoint['model'])
    model.load_state_dict(fixed_state_dict)
    
    return model

# 假设你的模型保存路径为
# model_path = '/hhd2/fan/SCR/output/PairwiseLecardBertBase_re/4.pth'
model_path = '/hhd2/fan/SCR/output/PairwiseLecardSAILER_visual/3.pth'
model = load_model(model_path)

/home/nlpc/miniconda3/envs/LED/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



using original BERT


Some weights of the model checkpoint at /hhd2/fan/SCR/Pretrain_model/SAILER were not used when initializing BertModel: ['cause_head.0.attention.self.query.weight', 'cause_head.0.attention.output.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'c_head.0.attention.output.dense.weight', 'cause_head.0.attention.self.value.bias', 'cls.predictions.transform.dense.bias', 'c_head.1.attention.self.key.bias', 'c_head.1.attention.self.value.weight', 'cause_head.0.intermediate.dense.weight', 'c_head.1.attention.self.query.weight', 'c_head.1.attention.output.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'c_head.0.attention.self.value.bias', 'c_head.1.attention.output.dense.weight', 'cause_head.0.attention.self.key.bias', 'c_head.1.attention.output.dense.bias', 'cause_head.0.output.dense.weight', 'c_head.0.attention.output.LayerNorm.weight', 'cause_head.0.output.LayerNorm.bias', 'c_head.0.output.dense.weight', 'c_head.0.attention.self.query.bias', 'cause_head.0.attention.ou

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

collate_fn = {}
formatter = {}
acc_result = None
data_path = "/hhd2/fan/SCR/SCR-Experiment/test.json"
def get_data(task_list, *args, **params):
    for task in task_list:
        # print(task)
        return PairwiseFormatter(task, *args, **params).process_single(data_path, "test")
data = get_data(["test"], data_path)
with torch.no_grad():
    logits, pooler_out, attention_weights = model(data, "valid", acc_result)
# print(attention_weights.shape)
attention_np = attention_weights.squeeze(0).numpy()
# print(attention_np.shape[0])
# print(attention_np.shape)
# 计算所有头的平均注意力权重
attention_avg = attention_np.mean(axis=0)
# 计算每个token的平均注意力权重
token_weights = attention_avg.max(axis=1)  # 可以选择max, mean等方式

# 归一化权重以便在颜色映射中使用
weights_normalized = token_weights / np.max(token_weights)
# 定义阈值为权重的75%分位数
threshold = np.percentile(weights_normalized, 79)

tokens = ['[CLS]', '2013', '年', '，', '被', '告', '人', '陈', '益', '桃', '受', '被', '告', '人', '邓', '某', '某', '指', '使', '，', '为', '解', '决', '邓', '与', '被', '害', '人', '杨', '某', '某', '之', '间', '的', '经', '济', '纠', '纷', '，', '纠', '集', '被', '告', '人', '瞿', '某', '某', '等', '人', '强', '行', '将', '被', '害', '人', '杨', '某', '某', '带', '至', '他', '处', '看', '管', '。', '期', '间', '，', '被', '告', '人', '陈', '益', '桃', '等', '人', '采', '用', '殴', '打', '及', '言', '语', '威', '胁', '等', '方', '式', '，', '逼', '迫', '杨', '某', '某', '签', '下', '悔', '过', '书', '并', '承', '诺', '放', '弃', '经', '济', '利', '益', '。', '被', '告', '人', '陈', '益', '桃', '等', '人', '又', '逼', '迫', '被', '害', '人', '杨', '某', '某', '出', '具', '欠', '条', '支', '付', '辛', '苦', '费', '5', '万', '元', '。', '[SEP]']
# tokens = ['[CLS]', '俞', '某', '某', '通', '过', '手', '机', '短', '信', '、', '[UNK]', '等', '形', '式', '共', '接', '收', '[UNK]', '六', '合', '彩', '[UNK]', '码', '单', '合', '计', '金', '额', '人', '民', '币', '480', '##84', '##8', '.', '18', '元', '，', '后', '将', '接', '收', '的', '[UNK]', '六', '合', '彩', '[UNK]', '码', '单', '报', '给', '上', '线', '庄', '家', '，', '从', '中', '非', '法', '获', '利', '。', '[SEP]']
# tokens = ['[CLS]', '原', '审', '被', '告', '人', '于', '守', '刚', '在', '任', '梅', '河', '口', '市', '农', '机', '局', '某', '某', '镇', '农', '机', '站', '站', '长', '期', '间', '，', '于', '2010', '年', '7', '月', '6', '日', '，', '在', '为', '本', '镇', '农', '民', '钟', '某', '某', '办', '理', '购', '买', '国', '家', '补', '贴', '农', '机', '时', '，', '向', '钟', '某', '某', '收', '取', '抵', '押', '金', '人', '民', '币', '2', '万', '元', '，', '并', '给', '钟', '某', '某', '出', '具', '加', '盖', '梅', '河', '口', '市', '某', '某', '镇', '农', '机', '工', '作', '站', '公', '章', '的', '收', '据', '1', '份', '，', '后', '将', '此', '款', '占', '为', '己', '有', '。', '案', '发', '后', '，', '原', '审', '被', '告', '人', '于', '守', '刚', '退', '还', '全', '部', '赃', '款', '。', '[SEP]']
# tokens = ['[CLS]','2010', '年', '5', '月', '17', '日', '17', '时', '许', '，', '李', '保', '庆', '（', '另', '案', '处', '理', '）', '以', '高', '某', '某', '调', '戏', '其', '外', '甥', '媳', '妇', '为', '由', '伙', '同', '被', '告', '人', '李', '爱', '群', '和', '张', '锋', '（', '另', '案', '处', '理', '）', '将', '高', '某', '某', '强', '行', '拉', '到', '安', '阳', '市', '小', '营', '村', '附', '近', '旅', '社', '内', '对', '其', '予', '以', '殴', '打', '，', '并', '拘', '禁', '至', '5', '月', '18', '日', '15', '时', '许', '，', '并', '且', '当', '场', '逼', '要', '高', '某', '某', '2000', '元', '现', '金', '，', '又', '让', '高', '某', '某', '书', '写', '了', '一', '张', '3000', '元', '的', '欠', '条', '。', '另', '查', '明', '，', '案', '发', '后', '被', '告', '人', '等', '已', '赔', '偿', '被', '害', '人', '经', '济', '损', '失', '2000', '元', '，', '并', '退', '赃', '款', '2000', '元', '。','[SEP]']
print(len(tokens))

assert len(tokens) == attention_np.shape[1], "tokens的长度与attention_weights不匹配"
# 创建HTML字符串以显示带有颜色编码的文本
# html_str = "<div style='font-size:16px;'>"
html_str = "<div style='font-size:16px; width: 450px; word-wrap: break-word;'>"
for token, weight in zip(tokens, weights_normalized):
    # print(weight.shape)
    # print(weight)
    if weight > threshold:
        alpha = weight  # 使用权重作为透明度
    else:
        alpha = 0.001 

    color = plt.get_cmap('magma')(alpha)[:3]  # 使用红色调色板，你可以选择你喜欢的颜色
    color = [int(x * 255) for x in color]
    # html_str += f"<span style='background-color: rgba({color[0]},{color[1]},{color[2]}, {alpha})'>{token}</span> "
    if token.isalpha():
        html_str += f"<span style='background-color: rgba({color[0]},{color[1]},{color[2]}, {alpha}); font-family: \"Times New Roman\", Times, serif;'>{token}</span> "
    else:
        html_str += f"<span style='background-color: rgba({color[0]},{color[1]},{color[2]}, {alpha})'>{token}</span> "
html_str += "</div>"

# 显示HTML字符串
display(HTML(html_str))
# 将HTML字符串保存到文件
with open('/hhd2/fan/SCR/SCR-Experiment/output_SAILER.html', 'w', encoding='utf-8') as file:
    file.write(html_str)

140


In [3]:
from transformers import BertTokenizer

# 初始化分词器
tokenizer = BertTokenizer.from_pretrained('/hhd2/fan/SCR/SCR-Preprocess/saved_BERT')

# 定义文本
text = "2013年，被告人陈益桃受被告人邓某某指使，为解决邓与被害人杨某某之间的经济纠纷，纠集被告人瞿某某等人强行将被害人杨某某带至他处看管。期间，被告人陈益桃等人采用殴打及言语威胁等方式，逼迫杨某某签下悔过书并承诺放弃经济利益。被告人陈益桃等人又逼迫被害人杨某某出具欠条支付辛苦费5万元。"
# text = "俞某某通过手机短信、QQ等形式共接收“六合彩”码单合计金额人民币480848.18元，后将接收的“六合彩”码单报给上线庄家，从中非法获利。"
# text = "2010年5月17日17时许，李保庆（另案处理）以高某某调戏其外甥媳妇为由伙同被告人李爱群和张锋（另案处理）将高某某强行拉到安阳市小营村附近旅社内对其予以殴打，并拘禁至5月18日15时许，并且当场逼要高某某2000元现金，又让高某某书写了一张3000元的欠条。另查明，案发后被告人等已赔偿被害人经济损失2000元，并退赃款2000元。"
# text = "原审被告人于守刚在任梅河口市农机局某某镇农机站站长期间，于2010年7月6日，在为本镇农民钟某某办理购买国家补贴农机时，向钟某某收取抵押金人民币2万元，并给钟某某出具加盖梅河口市某某镇农机工作站公章的收据1份，后将此款占为己有。案发后，原审被告人于守刚退还全部赃款。"
# 使用分词器将文本转换为词元
tokens = tokenizer.tokenize(text)

# 将分词结果按照指定的格式排列
formatted_tokens = ['[CLS]'] + tokens + ['[SEP]']

# 现在formatted_tokens包含了按照特定格式排列的词元
print(len(formatted_tokens))
print(formatted_tokens)

140
['[CLS]', '2013', '年', '，', '被', '告', '人', '陈', '益', '桃', '受', '被', '告', '人', '邓', '某', '某', '指', '使', '，', '为', '解', '决', '邓', '与', '被', '害', '人', '杨', '某', '某', '之', '间', '的', '经', '济', '纠', '纷', '，', '纠', '集', '被', '告', '人', '瞿', '某', '某', '等', '人', '强', '行', '将', '被', '害', '人', '杨', '某', '某', '带', '至', '他', '处', '看', '管', '。', '期', '间', '，', '被', '告', '人', '陈', '益', '桃', '等', '人', '采', '用', '殴', '打', '及', '言', '语', '威', '胁', '等', '方', '式', '，', '逼', '迫', '杨', '某', '某', '签', '下', '悔', '过', '书', '并', '承', '诺', '放', '弃', '经', '济', '利', '益', '。', '被', '告', '人', '陈', '益', '桃', '等', '人', '又', '逼', '迫', '被', '害', '人', '杨', '某', '某', '出', '具', '欠', '条', '支', '付', '辛', '苦', '费', '5', '万', '元', '。', '[SEP]']
